# Page 1

In [2]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/sunscreen-sun-care-tanning-skin-beauty/-/N-mpwpr?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    prev_count = 0
    same_count_times = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(3, 5))  # 更长的等待时间

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        curr_count = len(products)
        print(f"🧮 当前商品数: {curr_count}")

        if curr_count == prev_count:
            same_count_times += 1
        else:
            same_count_times = 0
            prev_count = curr_count

        if same_count_times >= 2:
            print("✅ 页面懒加载完成")
            break

        # 模拟人类行为滚动到底再抬头
        try:
            ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)



# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Sunscreen",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        old_url = driver.current_url
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)

        WebDriverWait(driver, 10).until(
            lambda d: d.current_url != old_url
        )
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮或 URL 未变化，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")


print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")


✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🧮 当前商品数: 16
🔄 滚动页面: 第 2 次
🧮 当前商品数: 16
🔄 滚动页面: 第 3 次
🧮 当前商品数: 16
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🧮 当前商品数: 0
🔄 滚动页面: 第 2 次
🧮 当前商品数: 0
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🧮 当前商品数: 0
🔄 滚动页面: 第 2 次
🧮 当前商品数: 0
✅ 页面懒加载完成
🔍 当前页面商品数: 29
🔁 检测到页面内容重复，疑似回到了第一页，停止抓取
⛔ 检测到重复页，主动中止爬虫
✅ WebDriver 关闭
📦 共抓取商品（去重后）：16 条


In [4]:
df = pd.DataFrame(product_list)
df

product_name                 brand  \
0   Blue Lizard Sensitive Mineral Sunscreen Lotion...           Blue Lizard   
1   Blue Lizard Kids' Mineral Sunscreen Stick for ...           Blue Lizard   
2   Vacation Sunscreen Classic Whip - SPF 30 - 4 f...              Vacation   
3     Sun Bum Original Sunscreen Spray - SPF 50 - 6oz               Sun Bum   
4   Black Girl Sunscreen Kids Broad Spectrum - SPF...  Black Girl Sunscreen   
5   Coola Dew Good Illuminating Sunscreen Serum - ...                 Coola   
6   Blue Lizard Baby Mineral Sunscreen Stick for F...           Blue Lizard   
7   Hero Force Shield Superlight Sunscreen SPF30 -...        Hero Cosmetics   
8   Vacation Orange Gelee Sunscreen Gel - SPF 30 -...              Vacation   
9   Coola Classic Sunscreen Body Spray - SPF 30 - ...                 Coola   
10  Coola Classic Sunscreen Face Lotion - SPF 50 -...                 Coola   
11  Coola Classic Sunscreen Body Spray - SPF 50 - ...                 Coola   
12  TULA SKINCARE Protect + Glow Daily Sunscreen G...         TULA SKINCARE   
13  Kopari Sun Shield Sunscreen - SPF 45 - Rose Go...                Kopari   
14  Kopari Sun Shield Body Glow - SPF 50 - Ulta Be...                Kopari   
15  Colorescience Sunforgettable Total Protection ...         Colorescience   

      current_price            unit_price  \
0            $18.29   ($3.66/fluid ounce)   
1             $9.99        ($19.98/ounce)   
2            $24.99         ($6.25/ounce)   
3            $20.99         ($3.50/ounce)   
4            $12.59   ($4.20/fluid ounce)   
5            $48.00        ($41.74/ounce)   
6            $12.19        ($24.38/ounce)   
7            $20.99  ($12.42/fluid ounce)   
8            $27.49         ($8.81/ounce)   
9            $28.00         ($4.06/ounce)   
10           $32.00        ($18.82/ounce)   
11           $28.00         ($4.67/ounce)   
12           $38.00  ($22.35/fluid ounce)   
13           $42.00         ($8.40/ounce)   
14  $18.00 - $42.00                   N/A   
15           $49.00                   N/A   

                             star_rating review_counts  is_on_sale  \
0   4.4 out of 5 stars with 2445 ratings  2445 reviews           0   
1    4.6 out of 5 stars with 692 ratings   692 reviews           0   
2    4.7 out of 5 stars with 473 ratings   473 reviews           0   
3    4.7 out of 5 stars with 666 ratings   666 reviews           0   
4    4.7 out of 5 stars with 844 ratings   844 reviews           0   
5    4.6 out of 5 stars with 375 ratings   375 reviews           0   
6    4.7 out of 5 stars with 332 ratings   332 reviews           0   
7   4.5 out of 5 stars with 1734 ratings  1734 reviews           0   
8    4.7 out of 5 stars with 316 ratings   316 reviews           0   
9    4.8 out of 5 stars with 269 ratings   269 reviews           0   
10  4.5 out of 5 stars with 3229 ratings  3229 reviews           0   
11   4.4 out of 5 stars with 100 ratings   100 reviews           0   
12  4.6 out of 5 stars with 6558 ratings  6558 reviews           0   
13   4.1 out of 5 stars with 391 ratings   391 reviews           0   
14   4.1 out of 5 stars with 548 ratings   548 reviews           0   
15     4.3 out of 5 stars with 4 ratings     4 reviews           1   

   is_free_shipping  is_3day_delivery product_category  \
0                 0                 1        Sunscreen   
1                 0                 1        Sunscreen   
2                 0                 1        Sunscreen   
3                 0                 1        Sunscreen   
4                 0                 1        Sunscreen   
5                 1                 1        Sunscreen   
6                 0                 1        Sunscreen   
7                 0                 1        Sunscreen   
8                 0                 1        Sunscreen   
9                 0                 1        Sunscreen   
10                0                 1        Sunscreen   
11                0                 1       

In [5]:
df.to_csv("sunscreen_data.csv",index=False)

# Page 2

In [4]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/sunscreen-sun-care-tanning-skin-beauty/-/N-mpwpr?Nao=24&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Sunscreen",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
🔁 检测到页面内容重复，疑似回到了第一页，停止抓取
⛔ 检测到重复页，主动中止爬虫
✅ WebDriver 关闭
📦 共抓取商品（去重后）：29 条


In [5]:
df2 = pd.DataFrame(product_list2)
df2

product_name            brand  \
0   Coppertone Every Tone Sunscreen Face Stick - S...       Coppertone   
1   La Roche Posay Anthelios, UV Correct Daily Ant...   La Roche Posay   
2   Neutrogena Beach Defense Sunscreen Lotion, SPF...       Neutrogena   
3   Eucerin Age Defense Face Sunscreen Lotion - SP...          Eucerin   
4             Kids' Sunscreen Spray - SPF 50 - up&up™            up&up   
5     Sun Bum Original Sunscreen Spray - SPF 30 - 6oz          Sun Bum   
6   La Roche Posay Mela B3 UV Face Sunscreen - SPF...   La Roche Posay   
7   Coppertone Hydrating Every Tone Face - SPF 60+...       Coppertone   
8   La Roche Posay Anthelios Melt in Milk Sunscree...   La Roche Posay   
9   Coppertone Pure & Simple Kid's Sunscreen Spray...       Coppertone   
10  Sun Bum Original Sunscreen Lotion - SPF 50 - 6...          Sun Bum   
11  Neutrogena Ultimate Sport Sunscreen Face Lotio...       Neutrogena   
12  Sun Bum Kids' Clear Sunscreen Spray - SPF 50 -...          Sun Bum   
13    Sun Bum Original Sunscreen Spray - SPF 70 - 6oz          Sun Bum   
14  Kid's Sport Sunscreen Stick - SPF 55 - 0.47oz/...            up&up   
15  Vacation Classic Sunscreen Spray - SPF 50 - 6 ...         Vacation   
16  CeraVe 100% Mineral Sunscreen Stick for Face a...           CeraVe   
17  thinkbaby Mineral Sunscreen Lotion SPF 50 - 6 ...        thinkbaby   
18     Sun Bum Sunscreen Face Stick - SPF 30 - 0.45oz          Sun Bum   
19  Blue Lizard Sensitive Mineral Sunscreen Lotion...      Blue Lizard   
20  Cetaphil Sheer Mineral Sunscreen Stick SPF 50,...         Cetaphil   
21  Neutrogena Ultra Sheer Moisturizing Face Sunsc...       Neutrogena   
22  Hawaiian Tropic Sheer Touch Ultra Radiance Lot...  Hawaiian Tropic   
23  Sun Bum Kids' Clear Sunscreen Face Stick - SPF...          Sun Bum   
24  Neutrogena Beach Defense Water Resistant Kids'...       Neutrogena   
25         Coppertone Sport Sunscreen Lotion - SPF 50       Coppertone   
26  Eucerin Sensitive Tinted Mineral Face Sunscree...          Eucerin   
27  Eucerin Age Defense Face Sunscreen Tinted Loti...          Eucerin   
28  Neutrogena Clear Face Liquid Sunscreen Lotion ...       Neutrogena   

      current_price            unit_price  \
0            $14.69  ($29.98/fluid ounce)   
1            $45.49  ($26.76/fluid ounce)   
2            $12.19   ($1.82/fluid ounce)   
3            $19.59   ($7.84/fluid ounce)   
4    $5.99 - $10.99                   N/A   
5            $20.99         ($3.50/ounce)   
6            $42.99                   N/A   
7            $17.69  ($10.41/fluid ounce)   
8            $32.99  ($11.00/fluid ounce)   
9            $14.69         ($2.94/ounce)   
10           $19.49   ($3.25/fluid ounce)   
11           $13.59   ($5.44/fluid ounce)   
12           $20.99         ($3.50/ounce)   
13           $20.99         ($3.50/ounce)   
14            $9.99        ($10.63/ounce)   
15           $22.49   ($3.75/fluid ounce)   
16           $10.99        ($23.38/ounce)   
17           $29.49   ($4.92/fluid ounce)   
18           $15.29        ($33.98/ounce)   
19           $18.29   ($3.66/fluid ounce)   
20            $9.99        ($19.98/ounce)   
21           $14.89   ($8.76/fluid ounce)   
22   $9.99 - $13.29                   N/A   
23           $15.29        ($28.85/ounce)   
24           $14.39         ($9.59/ounce)   
25           $11.79                   N/A   
26           $19.59  ($11.52/fluid ounce)   
27           $17.99   ($7.20/fluid ounce)   
28  $10.99 - $15.49                   N/A   

                             star_rating review_counts  is_on_sale  \
0    4.7 out of 5 stars with 193 ratings   193 reviews           0   
1    4.5 out of 5 stars with 951 ratings   951 reviews           0   
2    4.6 out of 5 stars with 723 ratings   723 reviews           0   
3    4.3 out of 5 stars with 547 ratings   547 reviews           0   
4   4.5 out of 5 stars with 1107 ratings  1107 reviews           0   
5    4.7 out of 5 stars with 467 ratings   467 re

In [6]:
df2.to_csv("sunscreen_data2.csv",index=False)

# Page 3

In [8]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/sunscreen-sun-care-tanning-skin-beauty/-/N-mpwpr?Nao=48&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Sunscreen",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
🔁 检测到页面内容重复，疑似回到了第一页，停止抓取
⛔ 检测到重复页，主动中止爬虫
✅ WebDriver 关闭
📦 共抓取商品（去重后）：29 条


In [9]:
df3 = pd.DataFrame(product_list3)
df3

product_name            brand  \
0   e.l.f. SKIN Suntouchable Invisible Sunscreen -...           e.l.f.   
1   Hawaiian Tropic Mineral Skin Nourishing Sunscr...  Hawaiian Tropic   
2   Coppertone Sport Sunscreen Spray - SPF 50 - 7....       Coppertone   
3   Hawaiian Tropic Mineral Skin Nourishing Sunscr...  Hawaiian Tropic   
4   Neutrogena Mineral Ultra Sheer Face and Body S...       Neutrogena   
5   Coppertone Sport Sunscreen Spray - SPF 30 - 7.3oz       Coppertone   
6   Vacation Classic Sunscreen Lotion - SPF 50 - 1...         Vacation   
7   Coppertone Brightening Every Tone Face Sunscre...       Coppertone   
8      Sun Bum Mineral Spray Sunscreen - SPF 30 - 6oz          Sun Bum   
9   Banana Boat Kids Sunscreen Roll On - SPF 60+ -...      Banana Boat   
10  Neutrogena Beach Defense Sunscreen Spray - SPF...       Neutrogena   
11  Naked Sundays Hydrating Glow Face Mist Top Up ...    Naked Sundays   
12  Neutrogena Beach Defense Sunscreen Spray, SPF ...       Neutrogena   
13  Babyganics Sheer Blend SPF 50 Mineral Sunscree...       Babyganics   
14  Neutrogena Invisible Daily Defense Sunscreen F...       Neutrogena   
15  Coppertone Pure & Simple Baby Mineral Sunscree...       Coppertone   
16  Babyganics Mineral-Based Baby Sunscreen Spray ...       Babyganics   
17  Neutrogena Beach Defense Sunscreen Lotion - SP...       Neutrogena   
18  Sun Bum Original Sunscreen Lotion - SPF 70 - 6...          Sun Bum   
19  La Roche Posay Anthelios, UV Correct Daily Ant...   La Roche Posay   
20  Vacation Classic Sunscreen Lotion - SPF 50 - 6...         Vacation   
21  Kopari Sunglaze Sheer Body Mist Sunscreen - SP...           Kopari   
22  Neutrogena Ultra Sheer Dry Touch Sunscreen Lot...       Neutrogena   
23       Sun Bum SPF 30 Scalp and Hair Mist - 2 fl oz          Sun Bum   
24  Babyganics Sheer Blend SPF 50 Mineral Sunscree...       Babyganics   
25             Sun Bum Face Mist - 3.4 fl oz - SPF 45          Sun Bum   
26   Sun Bum Sunscreen Face Lotion - SPF 50 - 3 fl oz          Sun Bum   
27  CeraVe Hydrating 100% Mineral Sunscreen for Fa...           CeraVe   
28  Coppertone Sport Sunscreen Spray - SPF 30 - Tw...       Coppertone   

   current_price            unit_price                            star_rating  \
0         $14.00   ($8.24/fluid ounce)  4.7 out of 5 stars with 10477 ratings   
1         $24.49       ($163.27/ounce)    3.9 out of 5 stars with 385 ratings   
2         $12.59         ($1.72/ounce)    4.6 out of 5 stars with 266 ratings   
3         $24.49       ($163.27/ounce)    3.9 out of 5 stars with 385 ratings   
4         $16.79  ($11.92/fluid ounce)    4.2 out of 5 stars with 326 ratings   
5         $12.59         ($1.72/ounce)    4.6 out of 5 stars with 148 ratings   
6          $6.99   ($6.99/fluid ounce)    4.7 out of 5 stars with 151 ratings   
7         $17.69  ($10.41/fluid ounce)    4.7 out of 5 stars with 162 ratings   
8         $21.49         ($3.58/ounce)    3.6 out of 5 stars with 146 ratings   
9         $10.99   ($4.40/fluid ounce)    3.7 out of 5 stars with 362 ratings   
10        $11.89         ($1.40/ounce)   4.5 out of 5 stars with 1335 ratings   
11        $22.99   ($9.07/fluid ounce)    4.8 out of 5 stars with 973 ratings   
12        $10.99         ($1.69/ounce)    4.6 out of 5 stars with 424 ratings   
13         $9.99   ($3.33/fluid ounce)    4.7 out of 5 stars with 155 ratings   
14        $13.49   ($7.94/fluid ounce)   4.4 out of 5 stars with 1333 ratings   
15        $14.99   ($2.50/fluid ounce)    4.2 out of 5 stars with 260 ratings   
16        $13.99   ($2.33/fluid ounce)      4 out of 5 stars with 303 ratings   
17         $2.49   ($2.49/fluid ounce)   4.6 out of 5 stars with 1161 ratings   
18        $19.49   ($3.25/fluid ounce)    4.7 out of 5 stars with 449 ratings   
19        $45.49  ($26.76/fluid ounce)    4.5 out of 5 stars with 951 ratings   
20        $20.49         ($3.42/ounce)    4.8 out of 5 stars with 492 ratings   
21        $39.00         ($8.30/ounce)

In [10]:
df3.to_csv("sunscreen_data3.csv",index=False)

# Page 4-6

In [11]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/sunscreen-sun-care-tanning-skin-beauty/-/N-mpwpr?Nao=72&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list4 = []

def scrape_products():
    global product_list4, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list4.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Sunscreen",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list4)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
⚠️ 页面跳转失败或未加载新内容
✅ WebDriver 关闭
📦 共抓取商品（去重后）：45 条


In [12]:
df4 = pd.DataFrame(product_list4)
df4

product_name                 brand  \
0   Hempz Sweet Pineapple & Honey Melon Face Herba...                 Hempz   
1   La Roche Posay Anthelios Clear Skin Dry Touch ...        La Roche Posay   
2   Eucerin Advanced Hydration Sunscreen Lotion - ...               Eucerin   
3   La Roche Posay Anthelios Sunscreen, Ultra-Ligh...        La Roche Posay   
4   La Roche Posay Anthelios Kids Gentle Lotion Su...        La Roche Posay   
5   La Roche Posay Anthelios Melt in Milk Sunscree...        La Roche Posay   
6   Coppertone Sport Sunscreen Spray - SPF 30 - 7.3oz            Coppertone   
7   Australian Gold Continuous Spray Sunscreen Wit...       Australian Gold   
8    Sun Bum Sunscreen Face Lotion - SPF 50 - 3 fl oz               Sun Bum   
9   Coppertone Sport Sunscreen Lotion - SPF 30 - 7...            Coppertone   
10  Blue Lizard Sensitive Mineral Sunscreen Spray ...           Blue Lizard   
11  e.l.f. SKIN Suntouchable All Set For Sun Sunsc...                e.l.f.   
12  Neutrogena Sheer Zinc Kids Sunscreen Lotion - ...            Neutrogena   
13  Black Girl Sunscreen Make it Hybrid with Zinc ...  Black Girl Sunscreen   
14  La Roche Posay Anthelios Tinted Ultra-Light Fl...        La Roche Posay   
15  Black Girl Sunscreen Broad Spectrum Infused wi...  Black Girl Sunscreen   
16  Native Mineral Sunscreen - Unscented - SPF 30 ...                Native   
17  Eucerin Advanced Hydration Sunscreen Spray - S...               Eucerin   
18  Banana Boat Kids' Sport Sunscreen Spray - SPF ...           Banana Boat   
19  Eucerin Face Oil Control Sunscreen Lotion - SP...               Eucerin   
20  Sun Bum Kids' Clear Sunscreen Lotion - SPF 50 ...               Sun Bum   
21  Australian Gold Sunscreen Spray Gel with Insta...       Australian Gold   
22  La Roche Posay Anthelios Tinted Face Sunscreen...        La Roche Posay   
23  e.l.f. SKIN Suntouchable Invisible Sunscreen -...                e.l.f.   
24  Cetaphil Sheer Mineral Liquid Face Sunscreen S...              Cetaphil   
25  Blue Lizard Kids' Mineral Sunscreen Stick for ...           Blue Lizard   
26  Cetaphil Sheer Mineral Sunscreen Face & Body S...              Cetaphil   
27  Banana Boat Dry Oil Clear Sunscreen Spray - SP...           Banana Boat   
28  Banana Boat Sport 360 Coverage Advanced Contro...           Banana Boat   
29  Hempz Sweet Pineapple & Honey Melon Face Herba...                 Hempz   
30  Australian Gold Sunscreen Spray Gel with Insta...       Australian Gold   
31  Native Mineral Sunscreen - Coconut & Pineapple...                Native   
32  Banana Boat Kids Sport Sunscreen Lotion - SPF ...           Banana Boat   
33  Coola Classic Sunscreen Travel Kit - 3pc - Ult...                 Coola   
34  Native Mineral Face Lotion - Unscented - SPF 3...                Native   
35  Bubble Skincare Plus One Tinted Mineral - SPF ...       Bubble Skincare   
36  e.l.f. Suntouchable Invisi-Stick Sunscreens - ...                e.l.f.   
37         Sun Bum Original Sunscreen Lotion - SPF 30               Sun Bum   
38  Hero Force Shield Superlight Sunscreen SPF30 -...        Hero Cosmetics   
39  CeraVe Hydrating Sheer Sunscreen Lotion for Fa...                CeraVe   
40  thinkbaby Safe Baby Sunscreen Spray and Sunscr...             thinkbaby   
41  Vacation Chardonnay Oil Sunscreen - SPF 30 - 3...              Vacation   
42  Hero Cosmetics Force Shield Apricot Sunscreen ...        Hero Cosmetics   
43  Adult Sport Sunscreen Stick - SPF 55 - 1.5oz -...                 up&up   
44     Sun Bum Mineral Spray Sunscreen - SPF 30 - 6oz               Sun Bum   

      current_price            unit_price  \
0            $12.99   ($4.33/fluid ounce)   
1   $25.49 - $26.99                   N/A   
2            $19.59   ($3.92/fluid ounce)   
3            $40.49  ($23.82/fluid ounce)   
4            $40.49   ($6.04/fluid ounce)   
5            $32.99  ($11.00/fluid ounce)   
6            $12.59         ($1.72/ounce)   
7   $11.89 - $12.59                   N/A   
8

In [13]:
df4.to_csv("sunscreen_data4.csv",index=False)

# 汇总第2-5页

In [15]:
df_combined_sunscreen_2_5 = pd.concat([df2,df3,df4], axis=0, ignore_index=True)
df_combined_sunscreen_2_5

product_name           brand  \
0    Coppertone Every Tone Sunscreen Face Stick - S...      Coppertone   
1    La Roche Posay Anthelios, UV Correct Daily Ant...  La Roche Posay   
2    Neutrogena Beach Defense Sunscreen Lotion, SPF...      Neutrogena   
3    Eucerin Age Defense Face Sunscreen Lotion - SP...         Eucerin   
4              Kids' Sunscreen Spray - SPF 50 - up&up™           up&up   
..                                                 ...             ...   
98   thinkbaby Safe Baby Sunscreen Spray and Sunscr...       thinkbaby   
99   Vacation Chardonnay Oil Sunscreen - SPF 30 - 3...        Vacation   
100  Hero Cosmetics Force Shield Apricot Sunscreen ...  Hero Cosmetics   
101  Adult Sport Sunscreen Stick - SPF 55 - 1.5oz -...           up&up   
102     Sun Bum Mineral Spray Sunscreen - SPF 30 - 6oz         Sun Bum   

      current_price            unit_price  \
0            $14.69  ($29.98/fluid ounce)   
1            $45.49  ($26.76/fluid ounce)   
2            $12.19   ($1.82/fluid ounce)   
3            $19.59   ($7.84/fluid ounce)   
4    $5.99 - $10.99                   N/A   
..              ...                   ...   
98           $37.49                   N/A   
99           $25.99   ($7.64/fluid ounce)   
100          $19.99  ($11.82/fluid ounce)   
101           $8.59         ($5.73/ounce)   
102          $21.49         ($3.58/ounce)   

                              star_rating review_counts  is_on_sale  \
0     4.7 out of 5 stars with 193 ratings   193 reviews           0   
1     4.5 out of 5 stars with 951 ratings   951 reviews           0   
2     4.6 out of 5 stars with 723 ratings   723 reviews           0   
3     4.3 out of 5 stars with 547 ratings   547 reviews           0   
4    4.5 out of 5 stars with 1107 ratings  1107 reviews           0   
..                                    ...           ...         ...   
98     4.5 out of 5 stars with 36 ratings    36 reviews           0   
99    4.7 out of 5 stars with 198 ratings   198 reviews           0   
100   4.6 out of 5 stars with 295 ratings   295 reviews           0   
101   4.4 out of 5 stars with 249 ratings   249 reviews           0   
102   3.6 out of 5 stars with 146 ratings   146 reviews           0   

    is_free_shipping  is_3day_delivery product_category  \
0                  0                 1        Sunscreen   
1                  1                 1        Sunscreen   
2                  0                 1        Sunscreen   
3                  0                 1        Sunscreen   
4                N/A                 1        Sunscreen   
..               ...               ...              ...   
98                 1                 0        Sunscreen   
99                 0                 1        Sunscreen   
100                0                 1        Sunscreen   
101                0                 1        Sunscreen   
102                0                 1        Sunscreen   

                                          product_link  
0    https://www.target.com/p/coppertone-every-tone...  
1    https://www.target.com/p/la-roche-posay-anthel...  
2    https://www.target.com/p/neutrogena-beach-defe...  
3    https://www.target.com/p/eucerin-age-defense-f...  
4    https://www.target.com/p/kids-sunscreen-spray-...  
..                                                 ...  
98   https://www.target.com/p/thinkbaby-safe-baby-s...  
99   https://www.target.com/p/vacation-chardonnay-o...  
100  https://www.target.com/p/hero-cosmetics-force-...  
101  https://www.target.com/p/adult-sport-sunscreen...  
102  https://www.target.com/p/sun-bum-mineral-spray...  

[103 rows x 11 columns]

In [16]:
df_combined_sunscreen_2_5.to_csv("sunscreen_2_5.csv",index=False)